# 階層的クラスタ分析

Iris（あやめ）を対象として階層的クラスタ分析

- 特徴量
  - sepal_length （がく片の長さ）
  - sepal_width （がく片の幅）
  - petal_length （花弁の長さ）
  - petal_width （花弁の幅）
- 種 (species) のカテゴリ
  - setosa ， versicolor ， virginica

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram

iris = sns.load_dataset('iris')

# 各種を確認
iris.iloc[[0, 50, 100]]

In [ ]:
# 散布図でデータの重なりを見る
sns.pairplot(iris, hue='species')

In [ ]:
# クラスタリング対象のデータから species 列を削除
X = iris.drop(['species'], axis=1)

# 階層的凝集クラスタリング
# - metric='euclidean' ユークリッド距離
# - method='single' 最小非類似度
clusters = linkage(X.values, method='single', metric='euclidean')

# 樹形図を作成
plt.figure(figsize=(50, 20))
dendr = dendrogram(clusters, labels=X.index, leaf_font_size=14)

In [ ]:
# - method='complete' 最大非類似度
plt.figure(figsize=(50, 20))
clusters = linkage(X.values, method='complete', metric='euclidean')
dendr = dendrogram(clusters, labels=X.index, leaf_font_size=14)

In [ ]:
# - method='average' 平均非類似度
plt.figure(figsize=(50, 20))
clusters = linkage(X.values, method='average', metric='euclidean')
dendr = dendrogram(clusters, labels=X.index, leaf_font_size=14)

### クラスタの類似度の違い

- 最小非類似度では、データを1つ1つ統合しながら、クラスタが大きくなる傾向がある。
- 一方、最大非類似度、平均非類似度では、クラスタが同程度の大きさになる傾向がある。

<hr>

### ヒートマップ付きの樹形図

ヒートマップは、数値の大小を色の濃さで表現する。

- 枝が最も長いところのヒートマップの濃淡の違い
- その下のクラスタのヒートマップの濃淡の違い

から、クラスタの切れ目が見えやすい。

In [ ]:
# - method='single' 最小非類似度
sns.clustermap(X, method='single', metric='euclidean', figsize=(14, 18),
               col_cluster=False)

In [ ]:
# - method='complete' 最大非類似度
sns.clustermap(X, method='complete', metric='euclidean', figsize=(14, 18),
               col_cluster=False)

In [ ]:
# - method='average' 平均非類似度
sns.clustermap(X, method='average', metric='euclidean', figsize=(14, 18),
               col_cluster=False)

In [ ]:
# - method='centroid' 重心法
sns.clustermap(X, method='centroid', metric='euclidean', figsize=(14, 18),
               col_cluster=False)

In [ ]:
# - method='ward' Ward法
sns.clustermap(X, method='ward', metric='euclidean', figsize=(14, 18),
               col_cluster=False)

<hr>

### クラスタ番号の取得

階層的クラスタ分析の結果を取得する。

In [ ]:
# linkage で距離の計算まで処理
# - metric='euclidean' ユークリッド距離
# - method='ward' ward法
row_clusters = linkage(X.values, method='ward', metric='euclidean')
# 樹形図を作成
plt.figure(figsize=(50, 20))
row_dendr = dendrogram(row_clusters, labels=X.index, leaf_font_size=14)

クラスタ数 3 のところの結果を取得する。

In [ ]:
from scipy.cluster.hierarchy import cut_tree
# cut_tree: 指定したクラスタ数でのラベル（クラスタ番号）を取得
# - np.ndarray.flatten: cut_tree は多次元配列で返すので、1次元配列に変換
labels = np.ndarray.flatten(cut_tree(row_clusters, 3))

# クロス集計により species と比較する。
pd.crosstab(labels, iris.species)

In [ ]:
# クラスタリング結果を散布図で描画
sns.scatterplot(x=X.petal_length, y=X.petal_width, hue=labels)

<hr>

### クラスタリング方法の違いによる結果の差を散布図で確認

クラスタ数は 3 に固定。

In [ ]:
methods = {'single': '最小非類似度',
           'complete': '最大非類似度',
           'average': '平均非類似度',
           'centroid': '重心法',
           'ward': 'Ward法'}

for m in methods:
    row_clusters = linkage(X.values, method=m, metric='euclidean')
    labels = np.ndarray.flatten(cut_tree(row_clusters, 3))
    print(methods[m])
    sns.scatterplot(x=X.petal_length, y=X.petal_width, hue=labels)
    plt.show()